<a href="https://colab.research.google.com/github/wksmirnowa/antidict/blob/master/deforming_words_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade gensim

     |████████████████████████████████| 24.2MB 67.8MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [2]:
!wget https://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz

--2020-05-15 13:47:36--  https://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2691248108 (2.5G) [application/x-gzip]
Saving to: ‘araneum_none_fasttextcbow_300_5_2018.tgz’

araneum_none_fastte 100%[===================>]   2.51G  20.8MB/s    in 2m 9s   

2020-05-15 13:49:46 (19.9 MB/s) - ‘araneum_none_fasttextcbow_300_5_2018.tgz’ saved [2691248108/2691248108]



In [0]:
import tarfile
fasttext_file = tarfile.open('/content/araneum_none_fasttextcbow_300_5_2018.tgz', 'r')
fasttext_file.extractall()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import gensim
import pandas as pd
import numpy as np

from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

In [0]:
model = gensim.models.KeyedVectors.load("araneum_none_fasttextcbow_300_5_2018.model")

In [0]:
data1 = pd.read_csv('/content/drive/My Drive/antidict_files/train_1.csv')

In [0]:
false_deform = pd.read_csv('/content/drive/My Drive/antidict_files/false_deforming.csv')

In [0]:
with open('/content/drive/My Drive/antidict_files/forms.csv') as file: # словарь словоформ
    forms1 = file.readlines()
forms1 = [word.strip('\"»«\n)(') for word in forms1]
forms1 = [word.lower() for word in forms1 if word]
labels = pd.Series(0, index=np.arange(len(forms1)))
forms1 = pd.DataFrame({'words': forms1, 'values': labels})

In [0]:
forms = pd.concat([forms1, false_deform], axis=0).sample(frac=1)
forms.drop(['Unnamed: 0'], axis=1, inplace=True)

In [0]:
df = pd.concat([forms.sample(1000), data1], axis=0).sample(frac=1)
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [0]:
df['model'] = df.words.apply(lambda x: model[x])

In [0]:
X = np.vstack(df.model.values)
y = df['values'].values

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [0]:
clf = LogisticRegression().fit(X_train, y_train)

In [16]:
y_pred = clf.predict(X_test)
f1_score(y_test, y_pred)

0.7277227722772276

In [25]:
from sklearn.ensemble import RandomForestClassifier
clf_RF = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_leaf=10, class_weight='balanced', random_state=42).fit(X_train, y_train)
y_pred = clf_RF.predict(X_test)
f1_score(y_test, y_pred)

0.7592592592592593

In [17]:
from sklearn import linear_model
clf_sgd = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
clf_sgd.fit(X, y)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [18]:
y_pred = clf_sgd.predict(X_test)
f1_score(y_test, y_pred)

0.7889447236180904

SGDClassifier показал лучшие результаты по f-мере, поэтому используем его для выявления искажений на всех данных

In [0]:
dataset = pd.read_csv('/content/drive/My Drive/antidict_files/words_only.csv', sep='\t', names=['word'])

In [20]:
%%time
ff = open('deforming.csv', 'w+')

for word in dataset.word:
    res = clf_sgd.predict(model[word].reshape(1,-1))
    if res[0] == 1:
        example = f"{word}\t{res}" + "\n"
        ff.write(example)


CPU times: user 20min 54s, sys: 1.26 s, total: 20min 55s
Wall time: 20min 56s


In [0]:
examples = pd.read_csv('deforming.csv', sep='\t', names=['word', 'class'])
examples.to_csv('deforming_1.csv')
